<a href="https://colab.research.google.com/github/Maks6666/ML/blob/main/%D0%94%D0%97_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [281]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [282]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/House_Rent_Dataset.csv", index_col="Unnamed: 0")

[Інформація про дані](https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset)

In [283]:
df.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2.0,10000.0,1100.0,Ground out of 2,NaN,NaN,Kolkata,Unfurnished,Bachelors/Family,2.0,Contact Owner
1,2022-05-13,2.0,20000.0,800.0,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1.0,NaN
2,2022-05-16,2.0,17000.0,1000.0,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1.0,Contact Owner
3,2022-07-04,2.0,10000.0,800.0,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1.0,Contact Owner
4,NaN,2.0,7500.0,850.0,1 out of 2,Carpet Area,NaN,Kolkata,Unfurnished,Bachelors,1.0,Contact Owner


In [284]:
df = df[['Rent', 'Size', 'Furnishing Status', 'City']]
df.head()


value = df['Rent'].mean()
df['Rent'].fillna(value, inplace=True)

# Завдання 1
Ввиведіть основні числові характеристики даних, розділіть на тренувальну та тестову чатини

In [285]:
df.isnull().sum()

Rent                   0
Size                 344
Furnishing Status    351
City                 358
dtype: int64

In [286]:
from sklearn.model_selection import train_test_split

# df.describe()
df.isnull().sum()

x = df.drop("Rent", axis="columns")
y = df["Rent"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [287]:
x

,Size,Furnishing Status,City
0,1100.0,Unfurnished,Kolkata
1,800.0,Semi-Furnished,Kolkata
2,1000.0,Semi-Furnished,Kolkata
3,800.0,Unfurnished,Kolkata
4,850.0,Unfurnished,Kolkata
...,...,...,...
4741,1000.0,Semi-Furnished,Hyderabad
4742,2000.0,Semi-Furnished,Hyderabad
4743,1750.0,Semi-Furnished,Hyderabad
4744,1500.0,Semi-Furnished,Hyderabad


# Завдання 2
Створіть Pipeline для обробки числових стовпчиків(заповнення пропусків)

In [288]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder



# num_col = x.select_dtypes(include="number").columns
# num_transform = Pipeline(steps=[("imputing", SimpleImputer(strategy="mean"))])


num_col = x.select_dtypes(include="number").columns
num_transform = Pipeline(steps=[
    ("imputing", SimpleImputer(strategy="mean"))
])



# Завдання 3
Створіть Pipeline для обробки категоріальних стовпчиків(заповнення пропусків та кодування)

In [289]:
# cat_col = x.select_dtypes(include="object").columns
# cat_transform = Pipeline(steps=[("ecnoding", OrdinalEncoder()),
#                                 ("imputing", SimpleImputer(strategy="mean"))
#                                ])


cat_col = x.select_dtypes(include="object").columns
cat_transform = Pipeline(steps=[
    ("encoding", OrdinalEncoder()),
    ("imputing", SimpleImputer(strategy="most_frequent"))
])


# Завдання 4
Об'єднайте попередні Pipeline в один ColumnTransformer

In [290]:
from sklearn.compose import ColumnTransformer

# preprocessor = ColumnTransformer(transformers=[("numerical", num_transform, num_col),
#                                                ("categorical", cat_transform, cat_col)])

preprocessor = ColumnTransformer(transformers=[
    ("numerical", num_transform, num_col),
    ("categorical", cat_transform, cat_col)
])

In [291]:
# preprocessor.fit(x_train)
# x_train = preprocessor.transform(x_train)

In [292]:
# x_train

# Завдання 5
Створіть остаточну модель та натренуйте її

In [293]:
from sklearn.linear_model import LinearRegression

# model = Pipeline(steps=[("preprocessor", preprocessor),
#                         ("model", LinearRegression())])


model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LinearRegression())
])

# Завдання 6
Виведіть основні метрики моделі

In [294]:
model.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputing',
                                                                   SimpleImputer())]),
                                                  Index(['Size'], dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoding',
                                                                   OrdinalEncoder()),
                                                                  ('imputing',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  Index(['Furnishing Status', 'City'], dtype='object'))])),
                ('model', LinearRegression())])

In [297]:
y_pred = model.predict(x_test)

In [299]:
from sklearn import metrics

def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()

  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df.round(2)



In [300]:
all_metrics = pd.DataFrame()

all_metrics["train"] = get_metrics(y_train, model.predict(x_train))
all_metrics["test"] = get_metrics(y_test, model.predict(x_test))

all_metrics


,train,test
MAE,25882.15,25344.85
RMSE,72323.20,42270.90
R2,0.17,0.27


In [295]:
data = pd.DataFrame({'Size': [900],
                     'Furnishing Status': ['Semi-Furnished'],
                     'City': ['Mumbai']})

data

,Size,Furnishing Status,City
0,900,Semi-Furnished,Mumbai


In [301]:
model.predict(data)

array([50759.41883294])